### Reprojecting Raster

In [ ]:
file_name='file address'

In [ ]:
old_ds=gdal.Open(file_name)

In [ ]:
old_ds.Getprojection()

In [ ]:
crs=osr.SpatialReference()

In [ ]:
crs.SetProjCS('UTM')
crs.SetUTM(44)
crs.SetWellKnownGeoCS('WGS84')

In [ ]:
crs.ExportToWkt()

In [ ]:
vrt_ds=gdal.AutoCreateWarpedVRT(old_ds,old_ds.GetProjection(),crs.ExportToWkt(),gdal.GRIORA_NearestNeighbour)

In [ ]:
plt.figure(figsize=(10,20))
plt.subplot(1,2,1)
plt.imshow(old_ds.ReadAsArray(),cmap='gray')
plt.subplot(1,2,2)
plt.imshow(vrt_ds.ReadAsArray(),cmap='gray')
#plt.colorbar()

In [ ]:
vrt_ds.Getprojection()

In [ ]:
vrt_ds=None
old_ds=None

### Mosaicing Images

>A mosaic is a combination or merge of two or more images

### GDAL Transformer

Creates to image transformer.

>This function creates a transformation object that maps from pixel/line coordinates on one image to pixel/line coordinates on another image. The images may potentially be georeferenced coordinates. This transformer potentially performs three concatenated transformations.

>> The first stage is from source image pixel/line coordinates to source image georeferenced coordinates

>> The second stage is to change projections from the source coordinates system to the destination coordinate system, assuming they differ.

>> The third stage is converting from destination image georeferenced coordinates to destination image coordinates.



In [ ]:
ds=gdal.Open(file_name)

In [ ]:
ds.GetProjection()

In [ ]:
tansform=gdal.Transformer(ds, None, [])

> TransformPoint(bDstToSrc, x, y, [z]}

> DstToSrc is a flag specifying if you want to compute offsets from the destination raster to the source raster or vice versa. Use True to go from the destination to the source and False to go to the other way.

In [ ]:
transform.TransformPoint(False,10,10)

In [ ]:
ds.GetGeoTansform()

In [ ]:
transform.TransformPoint(True, 82.99, 31.00)

### Mosaicing Images

> A mosaic is a combination or merge of two more images.

In [ ]:
base_dir=path(r'file path')

In [ ]:
os.chdir(base_dir)

In [ ]:
file_names=[image address]

In [ ]:
def calculate_extents(ds):
    trf=ds.GetGeoTransform()
    left=trf[0]
    right=trf[0]+trf[1]*ds.RasterXSize
    top=trf[3]
    bottom=trf[3]+trf[5]*ds.RasterYSize
    retrun(left, right, bottom, top)

In [ ]:
ds_list=list()
extents = []

In [ ]:
file_names

In [ ]:
for file in file_names:
    dss=gdal.Open(file)
    extent=calculate_extents(dss)
    extents.append(extent)
    ds_list.append(dss)

In [ ]:
extents

In [ ]:
extents=np.array(extents)

In [ ]:
extents

In [ ]:
mos_left=np.min(extents[:,0:2])

In [ ]:
mos_right=np.max(extents[:,0:2])

In [ ]:
mos_top=np.max(extents[:,2:])

In [ ]:
mos_bot=np.mix(extents[:,2:])

In [ ]:
print(mos_left, mos_right, mos_bot, mos_top)

In [ ]:
prj=dss.GetProjection()

In [ ]:
trf=dss.GetGeoTransform()

In [ ]:
mos_trf=list(trf)

In [ ]:
mos_trf

In [ ]:
mos_trf[0]=mos_left
mos_trf[3]=mos_top

In [ ]:
mos_trf

In [ ]:
driver=gdal.GetDriverByName('GTiff')

In [ ]:
width=math.ceil((mos_right-mos_left)/mos_trf[1])
height=math.ceil((mos_bot-mos_top)/mos_trf[5])

In [ ]:
width, height

In [ ]:
out_ds=driver.Create('mosaic_liss.tif', width, height, eType=gdal.GDT_UInt16)
out_ds.SetProjection(prj)
out_ds.SetGeoTansform(mos_trf)

In [ ]:
tran=gdal.Transform(ds_list[0],out_ds,[])

In [ ]:
suc, xyz  =tran.TransformPoint(False, 0, 0)

In [ ]:
xyz

In [ ]:
for ds in ds_list:
    tran=gdal.Tranform(ds, out_ds, [])
    suc, xyz =tran.TransformPoint(False, 0, 0)
    x,y,z=map(int, xyz)
    data_out=ds.GetRasterBand(1).ReadAsArray()
    out_ds.GetRasterBand(1).WriterArray(data_out, x, y)

In [ ]:
plt.figure(figsize=(10,10))
for i in range(l, len(ds_list)+1):
    plt.subplot(2,2,i)
    plt.imshow(ds_list[i-1].ReadAsArray(),cmap='gray',clim=(150,150),)

In [ ]:
mosaic_data=out_ds.ReadAsArray()

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(mosaic_data,cmap='gray')

In [ ]:
out_ds.BuildOverviews('average', [2,4,8,16,32])

In [ ]:
out_ds.FlushCache()
del ds_list

In [ ]:
del out_ds

In [ ]:
plt.close()

### Local analyses on image

> Local analysis hey work on two or more arrays that are the same size, and an algebric equation is applied to each set of pixel locations. One common task using multispectral imagery is to compute various indices for tasks such as distinguishing between burned and unburned land or measuring nitrogen contained in vegetation. Let's took at an index used to measure "greenness," the normalized difference vegetation index(NDVI). The NDVI is a simple index that uses red and near-infrared wavelengths to produce a number that ranges from -1 to 1. Growing plants use red wavelengts for photosynthesis, but reflect near-infrared radiation, so a high ration of these two measurements can indicate photosynthetic activity and healthy vegetation.

### NDVI from image

### NDVI = NIR-RED/NIR+RED

In [ ]:
base_dir=path(r'directory path')

In [ ]:
os.chdir(base_dir)

In [ ]:
nir_image=''
red_image=''

In [ ]:
red_ds = gdal.Open(red_image)

In [ ]:
nir_ds=gdal.Open(nir_image)

In [ ]:
red_band=red_ds.GetRasterBand(1)
print(gdal.GetDataTypeName(red_band.DataType))

In [ ]:
nir_band=nir_ds=GetRasterBand(1)
print(gdal.GetDataTypeName(nir_band.DataType))

In [ ]:
red=red_band.ReadAsArray().astype(np.int32)

In [ ]:
nir=nir_band.ReadAsArray().astype(np.int32)

In [ ]:
ndvi=(nir-red)/(nir+red)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(ndvi,cmap='gray')
plt.colorbar()

### NDWI Calculation

### NDWI = GREEN-NIR/GREEN+NIR

In [ ]:
green_image='image address'
nir_image='image address'

In [ ]:
green=gdal.Open(green_image).ReadAsArray().astype(np.int32)
nir=gdal.Open(nir_image).ReadAsArray().astype(np.int32)

In [ ]:
ndwi=(green-nir)/(green+nir)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(ndwi,cmap='gray')
plt.colorbar()

In [ ]:
ndwi = np.where(ndwi>0.1,1,0)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(ndwi,cmap='gray')
plt.colorbar()

In [ ]:
import numpy.ma as ma

In [ ]:
water_mask=ma.masked_less(ndwi,0.1)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(water_mask,cmap='gray')
plt.colorbar()

In [ ]:
del green
del nir

### Global analyses

>Global functions work on the entire image, such as proximity analysis or cost distance. A proximity analysis determines the Euclidean distance of each cell to the nearest cell that's marked as a source

In [ ]:
water_layer='image name'

In [ ]:
ds=gdal.Open(water_layer)

In [ ]:
band=ds.GetRasterBand(1)

In [ ]:
water=band.ReadAsArray()

In [ ]:
water=np.where(water>1,1,0)

In [ ]:
band.WriteArray(water)

In [ ]:
plt.imshow(water,cmap='gray')
plt.colorbar()

In [ ]:
h,w=water.shape

In [ ]:
drv=gdal.GetDriverByName('gtiff')

In [ ]:
dst_ds=drv.Create('promximity.tif', w, h, 1)

In [ ]:
dst_ds.SetProjection(ds.GetProjection())

In [ ]:
dst_ds.SetGeoTransform(ds.GetGeoTransform())

In [ ]:
gdal.ComputerProximity(band,dst_ds.GetRasterBand(1),['DISTUNITS = GEO'])

In [ ]:
plt.figure(figsize=(15,20))
plt.subplot(1,2,1)
plt.imshow(dst_ds.ReadAsArray(),cmap='gray')

plt.subplot(1,2,2)
plt.imshow(band.ReadAsArray(),cmap='gray')

In [ ]:
dst_ds.FlushCache()

In [ ]:
dst_ds=None

### DEM Processing

>The Hillshade obtains the hypothetical illumination of a surface by determining illumination values for each cell in a raster. It does this by setting a position for a hypothetical light source and calculating the illumination values of each cell in realition to neighboring cells. It can greatly enhance the visualization of a surface for analysis or graphical display, especially when using transparency.

In [ ]:
dem='address'

In [ ]:
#opt=gdal.DEMProcessingOptions(slopeformat='degree')

In [ ]:
kk=gdal.DEMProcessing('hill.tif',dem,processing='hillshade')
del kk

In [ ]:
slope=gdal.Open('hill.tif')

In [ ]:
plt.figure(figsize=(15,20))
plt.imshow(slope.ReadAsArray(),cmap='terrain')
#plt.colorbar()

In [ ]:
plt.close()
del slope